In [12]:
# 1. Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import joblib
import os




In [6]:
# 2. Load dataset
df = pd.read_csv("data/processed/processed_gait_dataset.csv", header=None)
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values
print(f"Shape: {X.shape}, Labels: {np.unique(y)}")

Shape: (1801, 66), Labels: [0 1]


In [7]:
# 3. Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [8]:

# 4. Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# 5. Build model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


c:\Users\jayso\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# 6. Train
history = model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=32,
                    callbacks=[early_stopping], verbose=1)

In [ ]:
# 7. Evaluate
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc:.4f}")

In [ ]:
# 8. Save
os.makedirs("models", exist_ok=True)
model.save("models/gait_classifier.keras")
joblib.dump(scaler, "models/scaler.pkl")